In [1]:
import os
from tensorflow.keras.optimizers import Adam
import wandb
from keras.models import load_model
from utils.ModelWrapper import ModelWrapper, ModelType

In [2]:
config_defaults = {
    'epochs': 30,
    'batch_size': 32,
    'learning_rate': 0.0001,
    'dropout': 0.5,
    'regularization': 0.0001,
}
wandb.init(config=config_defaults, project="deepfake-resnet", entity="dat550")
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dat550 (use `wandb login --relogin` to force relogin)


In [3]:
img_size = 128
batch_size = config.batch_size
data_dir = "./data/data-large/data"

In [4]:
class_names=os.listdir(f'{data_dir}/train')
class_names

['.DS_Store', 'FAKE', 'REAL']

In [5]:
model_file = f'models/resnet_{wandb.run.name}_model.h5'


# init wrapper
modeltype = ModelType.RESNET
model_wrapper = ModelWrapper(data_dir, img_size, config, model_file=model_file, modeltype=modeltype, sample_datasets=True) # TODO: REMOVE SAMPLe

model_wrapper.model.compile(optimizer=Adam(learning_rate=config.learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model_wrapper.model.summary()


Found 29988 images belonging to 2 classes.
Found 3431 images belonging to 2 classes.
Found 1587 images belonging to 2 classes.


wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 2048)              58370944  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 59,485,825
Trainable params: 59,334,401
Non-trainable params: 151,424
_________________________________________________________________


In [ ]:
history = model_wrapper.fit()
print(history.history)

### Evaluate

In [ ]:
# load the saved model that is considered the best
eval_wandb_model = False
if eval_wandb_model:
    # load the bess model NB: edit namd and path
    loaded_model = wandb.restore('model-best.h5', run_path="dat550/deepfake-efficientnet/runs/29425ckm")
    model_to_eval = load_model(loaded_model.name)
else:
    model_to_eval = model_wrapper.model

In [ ]:
model_wrapper.evaluate_model(model_to_eval)

In [ ]:
model_wrapper.export_to_png()